In [2]:
#importing libraries:
import requests
import itertools
import pandas as pd
from bs4 import BeautifulSoup
from pandas import DataFrame as df 
import re

In [3]:
# # #Reading the URL's
URLs = pd.read_csv('Read_CBI_URLyy.csv')
URLs_list = pd.DataFrame(URLs)
# print (URL_column)
count_before = len(URLs_list)
print ('The database has', count_before,  'URL listed')
# f = requests.get ('https://www.cbi.ca/web/physiotherapy-belleville-millennium')
# soup = BeautifulSoup (f.text, 'html.parser')

The database has 8 URL listed


In [4]:
URLs_list_cleaned = URLs_list.drop_duplicates(subset=['URLs'])
URLs_list_cleaned.count()
URL_column = URLs_list_cleaned['URLs']
count_after = len(URL_column)
print ('There is', count_after,  'URL listed after removing the duplicates')


There is 8 URL listed after removing the duplicates


In [6]:
############################################### Reading the Location ID's #####################################################
Loc_ID = pd.read_excel('Read_CBI_Locations_ID.xlsx')
Location_ID = pd.DataFrame(Loc_ID)

In [7]:
############################################### Reading the Job Titles DB #####################################################
Job_Titles_DB = pd.read_excel('Read_Job_Titles_CBI_to_SunLife.xlsx')
JT_CBI_SunLife = pd.DataFrame(Job_Titles_DB)
# JT_CBI_SunLife

In [8]:
############################################### Reading the Services DB #####################################################
Services_DB = pd.read_excel('Read_Services_CBI_to_SunLife.xlsx')
Services_CBI_SunLife = pd.DataFrame(Services_DB)
# Services_CBI_SunLife

In [9]:
   ########################################## Creating all of the Functions ################################################

#Extracting the Location Name
def Location_Name():
    Locationportlet = soup.find_all('div', attrs = {'class' : 'p-locationaddress2-address'})
    # print(Locationportlet)
    for i in Locationportlet:
        Location = soup.find('h2').text
    # print (Location)
    return Location


#Extracting the Location Name 
def URL():
    Locationportlet = soup.find_all('div', attrs = {'class' : 'p-locationaddress2-address'})
    for i in Locationportlet:
        URL = soup.find_all('link')[2]["href"]
        return URL


##Extracting Fax:
def Fax():
    Locationportlet = soup.find_all('div', attrs = {'class' : 'p-locationaddress2-address'})
    for i in Locationportlet:
        Location_sentence = i.get_text(separator='br/')
        try:
            Fax_start = Location_sentence.find('fax') - 13
            Fax_end = Fax_start + 12
        #     print (Fax_start)
        #     print (Fax_end)
            Fax = Location_sentence[Fax_start:Fax_end]

        except:
            Fax = "Not Informed"
        # print (Fax)
        return Fax
    

##Extracting Phone:
def Phone_number():
    Locationportlet = soup.find_all('div', attrs = {'class' : 'p-locationaddress2-address'})
    for i in Locationportlet:
        Location_sentence = i.get_text(separator='br/')
        Comma_index = Location_sentence.rfind(',')
        # print (Phone_source)
        Phone_start = Comma_index + 21
        Phone_end = Phone_start + 12
        Phone = Location_sentence[Phone_start:Phone_end]
        # print (Phone)
        return Phone

    
##Extracting Postal Code:
def Postal_code():
    Locationportlet = soup.find_all('div', attrs = {'class' : 'p-locationaddress2-address'})
    
    for i in Locationportlet:
        Location_sentence = i.get_text(separator='br/')
        Comma_index = Location_sentence.rfind(',')
        Postal_code_start = Comma_index + 9
        Postal_code_end = Postal_code_start + 7
        Postal_code = Location_sentence[Postal_code_start:Postal_code_end]
        # print (Postal_code)
        return Postal_code

    
##Extracting Province :
def Province():
    Locationportlet = soup.find_all('div', attrs = {'class' : 'p-locationaddress2-address'})
#     print (Locationportlet)

    for i in Locationportlet:
        Location_sentence = i.get_text(separator='br/')
        Comma_index = Location_sentence.rfind(',')
        Province_start = Comma_index + 2
        Province_end = Province_start + 2
        Province = Location_sentence[Province_start:Province_end]
        # print (Province)
        return Province


##Extracting City:
def City():
    Locationportlet = soup.find_all('div', attrs = {'class' : 'p-locationaddress2-address'})
#     print (Locationportlet)

    for i in Locationportlet:
        Location_sentence = i.get_text(separator='br/')
        Comma_index = Location_sentence.rfind(',')
        sentence_until_comma = Location_sentence[:Comma_index]
        City_start = sentence_until_comma.rfind('br/') + 4
        # print (City_start)
        City = sentence_until_comma[City_start:]
        # print (City)
        return City

    
##Extracting Address_1:
def Address_1():
    Locationportlet = soup.find_all('div', attrs = {'class' : 'p-locationaddress2-address'})
#     print (Locationportlet)
    Location_final_index = len(Location_Name())
    for i in Locationportlet:
        Location_sentence = i.get_text(separator='br/')
        Address_1_start = Location_final_index + 8
        Address_1_end = Location_sentence.find('br/',Address_1_start)
        # print (Address_1_start)
        # print (Address_1_end)
        Address_1 = Location_sentence [Address_1_start:Address_1_end]
        # print (Address_1)
        return Address_1

    
##Extracting Address_2:
def Address_2():
    Locationportlet = soup.find_all('div', attrs = {'class' : 'p-locationaddress2-address'})
#     print (Locationportlet)
    Location_final_index = len(Location_Name())
    for i in Locationportlet:
        Location_sentence = i.get_text(separator='br/')
        Address_1_start = Location_final_index + 8
        Address_1_end = Location_sentence.find('br/',Address_1_start)
        Address_2_start = Address_1_end + 4
        Address_2_end = Location_sentence.find('br/',Address_2_start)-1
#     print (Address_2_start)
#     print (City_start)
    Comma_index = Location_sentence.rfind(',')
    sentence_until_comma = Location_sentence[:Comma_index]
    City_start = sentence_until_comma.rfind('br/') + 4
    if Address_2_start == City_start:
        Address_2 = '----'
    else:
        Address_2 = Location_sentence [Address_2_start:Address_2_end]
    return Address_2


##Extracting features:
def Features ():
    Features_list = []
    for i in soup.find_all('div', attrs = {'class', 'p-locationfeatures2-feature'}):
        Features_list.append(i.text)
#     print (*Features, sep=", ")
    Feature_1 = ";".join(Features_list)
#     print (Feature_1)
    return Feature_1


##Extracting Services considering SunLife Database:
def Services():
    Services_source = []
    Service_list_1 = []
    for i in soup.find_all('div', attrs = {'class', 'row-fluid p-locationservices-bizservices-row'}):
        Services_source.append (i.get_text(separator=';'))

    if len(Services_source) == 2:
        Services_source_1 = Services_source [0] + Services_source [1]
#         print (Services_source_1)
    else:
        Services_source_1 = ''.join(Services_source)

    # print (Services_source_1)

    Services_1 = re.sub('; ;', ';', Services_source_1)
    # print (Services_1)
    Services_2 = re.sub('; ;', ';', Services_1)
    # print (Services_2)
    Services_3 = re.sub(';  ;', ';', Services_2)
    # print (Services_3)
    Services_4 = re.sub('; ;', ';', Services_3)
    # print (Services_4)
    Services = Services_4[Services_4.find(';')+1:Services_4.rfind(';')]
    
    Services_1 = Services.split (";")
    Services_list = []
    Services_Dataframe = pd.DataFrame(Services_1)
    Services_Dataframe.columns = ['Services_CBI']
    Services_Dataframe.set_index('Services_CBI', inplace = True , drop = True)
    # Job_Title_final = Job_Title_SunLife.loc [Job, 'SunLife_Titles']
    Services_Dataframe

    Services_SunLife_Crossed = Services_Dataframe.merge (Services_CBI_SunLife, left_on='Services_CBI', right_on='CBI_Services', how='left')
    Cleaned_dataframe = Services_SunLife_Crossed.loc[Services_SunLife_Crossed['Sun_Life_Services']!= 'REVISE']
    Services_to_use = Cleaned_dataframe ['Sun_Life_Services']
    Services_to_use_1 = Services_to_use.drop_duplicates()
    Services_to_use_1
    
    for i in Services_to_use_1:
        Services_list.append(i)
#     print (Services_list)
    
    Services_list_1 = ";".join(Services_list)
#         for i in Services_list:
#             Service_list_1.append(i)
        
    return Services_list_1



#Extracting Side Content
def Side_Content():
    try:
        for i in soup.find_all('div', attrs = {'class' : 'portlet-layout row-fluid'}):
            for p in soup.find_all('div', attrs = {'id' : 'column-6'}):
                Side_Content_Full_Sentence = p.text
            break    

        Side_Content_Full_Sentence_end = Side_Content_Full_Sentence.find('Liferay.Analytics')
        # print (Side_Content_Full_Sentence_end)
        Side_Content = Side_Content_Full_Sentence [7:Side_Content_Full_Sentence_end-4]
        return Side_Content
    except:
        Side_Content = "Not Informed" 
        return Side_Content


#Extracting monday opening hour
def Monday_open():
    Hours_list = []
    Hoursportlet = soup.find_all('div', attrs = {'class' : 'p-locationfeatures2-main'})
    # print(Hoursportlet)
    Hour_open_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
        Hour_open = h.text [ :h.text.find('-')-1]
    #   Hour_close = h.text [ h.text.find('-')+2 : ]
        Hour_open_list.append (Hour_open)

    WeekDay_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
    #       Hour_open = h.text [ :h.text.find('-')-1]
    #       Hour_close = h.text [ h.text.find('-')+2 : ]
        WeekDay_list.append (s.text)
    WeekDay_list

    if 'Monday' in WeekDay_list:
        monday = 'Monday'
    else:
        monday = 'No' 

    if monday == 'Monday':
        monday_open = Hour_open_list[0]
        return monday_open
    else:
        monday_open = 'Close'
        return monday_open

    
#Extracting monday closing hour    
def Monday_close():
    Hours_list = []
    Hoursportlet = soup.find_all('div', attrs = {'class' : 'p-locationfeatures2-main'})
    # print(Hoursportlet)
    Hour_close_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
        Hour_close = h.text [ h.text.find('-')+2 : ]
        Hour_close_list.append (Hour_close)     

    WeekDay_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
    #       Hour_open = h.text [ :h.text.find('-')-1]
    #       Hour_close = h.text [ h.text.find('-')+2 : ]
        WeekDay_list.append (s.text)
    WeekDay_list

    if 'Monday' in WeekDay_list:
        monday = 'Monday'
    else:
        monday = 'No' 

    if monday == 'Monday':
        monday_close = Hour_close_list[0]
        return monday_close
    else:
        monday_close = 'Close'
        return monday_close


    
#Extracting Tuesday opening hour
def Tuesday_open():
    Hours_list = []
    Hoursportlet = soup.find_all('div', attrs = {'class' : 'p-locationfeatures2-main'})
    # print(Hoursportlet)
    Hour_open_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
        Hour_open = h.text [ :h.text.find('-')-1]
    #   Hour_close = h.text [ h.text.find('-')+2 : ]
        Hour_open_list.append (Hour_open)

    WeekDay_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
    #       Hour_open = h.text [ :h.text.find('-')-1]
    #       Hour_close = h.text [ h.text.find('-')+2 : ]
        WeekDay_list.append (s.text)
    WeekDay_list

    if 'Tuesday' in WeekDay_list:
        tuesday = 'Tuesday'
    else:
        tuesday = 'No' 

    if tuesday == 'Tuesday':
        tuesday_open = Hour_open_list[1]
        return tuesday_open
    else:
        tuesday_open = 'Close'
        return tuesday_open


#Extracting Tuesday closing hour       
def Tuesday_close():
    Hours_list = []
    Hoursportlet = soup.find_all('div', attrs = {'class' : 'p-locationfeatures2-main'})
    # print(Hoursportlet)
    Hour_close_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
        Hour_close = h.text [ h.text.find('-')+2 : ]
        Hour_close_list.append (Hour_close)     

    WeekDay_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
    #       Hour_open = h.text [ :h.text.find('-')-1]
    #       Hour_close = h.text [ h.text.find('-')+2 : ]
        WeekDay_list.append (s.text)
    WeekDay_list

    if 'Tuesday' in WeekDay_list:
        tuesday = 'Tuesday'
    else:
        tuesday = 'No' 

    if tuesday == 'Tuesday':
        tuesday_close = Hour_close_list[1]
        return tuesday_close
    else:
        tuesday_close = 'Close'
        return tuesday_close


    
#Extracting Wednesday opening hour
def Wednesday_open():
    Hours_list = []
    Hoursportlet = soup.find_all('div', attrs = {'class' : 'p-locationfeatures2-main'})
    # print(Hoursportlet)
    Hour_open_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
        Hour_open = h.text [ :h.text.find('-')-1]
    #   Hour_close = h.text [ h.text.find('-')+2 : ]
        Hour_open_list.append (Hour_open)

    WeekDay_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
    #       Hour_open = h.text [ :h.text.find('-')-1]
    #       Hour_close = h.text [ h.text.find('-')+2 : ]
        WeekDay_list.append (s.text)
    WeekDay_list

    if 'Wednesday' in WeekDay_list:
        wednesday = 'Wednesday'
    else:
        wednesday = 'No' 

    if wednesday == 'Wednesday':
        wednesday_open = Hour_open_list[2]
        return wednesday_open
    else:
        wednesday_open = 'Close'
        return wednesday_open


#Extracting Wednesday closing hour
def Wednesday_close():
    Hours_list = []
    Hoursportlet = soup.find_all('div', attrs = {'class' : 'p-locationfeatures2-main'})
    # print(Hoursportlet)
    Hour_close_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
        Hour_close = h.text [ h.text.find('-')+2 : ]
        Hour_close_list.append (Hour_close)     

    WeekDay_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
    #       Hour_open = h.text [ :h.text.find('-')-1]
    #       Hour_close = h.text [ h.text.find('-')+2 : ]
        WeekDay_list.append (s.text)
    WeekDay_list

    if 'Wednesday' in WeekDay_list:
        wednesday = 'Wednesday'
    else:
        wednesday = 'No' 

    if wednesday == 'Wednesday':
        wednesday_close = Hour_close_list[2]
        return wednesday_close
    else:
        wednesday_close = 'Close'
        return wednesday_close


    
#Extracting Thursday opening hour
def Thursday_open():
    Hours_list = []
    Hoursportlet = soup.find_all('div', attrs = {'class' : 'p-locationfeatures2-main'})
    # print(Hoursportlet)
    Hour_open_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
        Hour_open = h.text [ :h.text.find('-')-1]
    #   Hour_close = h.text [ h.text.find('-')+2 : ]
        Hour_open_list.append (Hour_open)

    WeekDay_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
    #       Hour_open = h.text [ :h.text.find('-')-1]
    #       Hour_close = h.text [ h.text.find('-')+2 : ]
        WeekDay_list.append (s.text)
    WeekDay_list

    if 'Thursday' in WeekDay_list:
        thursday = 'Thursday'
    else:
        thursday = 'No' 

    if thursday == 'Thursday':
        thursday_open = Hour_open_list[3]
        return thursday_open
    else:
        thursday_open = 'Close'
        return thursday_open


#Extracting Thursday closing hour    
def Thursday_close():
    Hours_list = []
    Hoursportlet = soup.find_all('div', attrs = {'class' : 'p-locationfeatures2-main'})
    # print(Hoursportlet)
    Hour_close_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
        Hour_close = h.text [ h.text.find('-')+2 : ]
        Hour_close_list.append (Hour_close)     

    WeekDay_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
    #       Hour_open = h.text [ :h.text.find('-')-1]
    #       Hour_close = h.text [ h.text.find('-')+2 : ]
        WeekDay_list.append (s.text)
    WeekDay_list

    if 'Thursday' in WeekDay_list:
        thursday = 'Thursday'
    else:
        thursday = 'No' 

    if thursday == 'Thursday':
        thursday_close = Hour_close_list[3]
        return thursday_close
    else:
        thursday_close = 'Close'
        return thursday_close


#Extracting Friday opening hour      
def Friday_open():
    Hours_list = []
    Hoursportlet = soup.find_all('div', attrs = {'class' : 'p-locationfeatures2-main'})
    # print(Hoursportlet)
    Hour_open_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
        Hour_open = h.text [ :h.text.find('-')-1]
    #   Hour_close = h.text [ h.text.find('-')+2 : ]
        Hour_open_list.append (Hour_open)

    WeekDay_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
    #       Hour_open = h.text [ :h.text.find('-')-1]
    #       Hour_close = h.text [ h.text.find('-')+2 : ]
        WeekDay_list.append (s.text)
    WeekDay_list

    if 'Friday' in WeekDay_list:
        friday = 'Friday'
    else:
        friday = 'No' 

    if friday == 'Friday':
        friday_open = Hour_open_list[4]
        return friday_open
    else:
        friday_open = 'Close'
        return friday_open


    
#Extracting Friday closing hour     
def Friday_close():
    Hours_list = []
    Hoursportlet = soup.find_all('div', attrs = {'class' : 'p-locationfeatures2-main'})
    # print(Hoursportlet)
    Hour_close_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
        Hour_close = h.text [ h.text.find('-')+2 : ]
        Hour_close_list.append (Hour_close)     

    WeekDay_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
    #       Hour_open = h.text [ :h.text.find('-')-1]
    #       Hour_close = h.text [ h.text.find('-')+2 : ]
        WeekDay_list.append (s.text)
    WeekDay_list

    if 'Friday' in WeekDay_list:
        friday = 'Friday'
    else:
        friday = 'No' 

    if friday == 'Friday':
        friday_close = Hour_close_list[4]
        return friday_close
    else:
        friday_close = 'Close'
        return friday_close


#Extracting Saturday opening hour     
def Saturday_open():
    Hours_list = []
    Hoursportlet = soup.find_all('div', attrs = {'class' : 'p-locationfeatures2-main'})
    # print(Hoursportlet)
    Hour_open_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
        Hour_open = h.text [ :h.text.find('-')-1]
    #   Hour_close = h.text [ h.text.find('-')+2 : ]
        Hour_open_list.append (Hour_open)

    WeekDay_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
    #       Hour_open = h.text [ :h.text.find('-')-1]
    #       Hour_close = h.text [ h.text.find('-')+2 : ]
        WeekDay_list.append (s.text)
    WeekDay_list

    if 'Saturday' in WeekDay_list:
        saturday = 'Saturday'
    else:
        saturday = 'No' 

    if saturday == 'Saturday':
        saturday_open = Hour_open_list[5]
        return saturday_open
    else:
        saturday_open = 'Close'
        return saturday_open


#Extracting Saturday closing hour
def Saturday_close():
    Hours_list = []
    Hoursportlet = soup.find_all('div', attrs = {'class' : 'p-locationfeatures2-main'})
    # print(Hoursportlet)
    Hour_close_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
        Hour_close = h.text [ h.text.find('-')+2 : ]
        Hour_close_list.append (Hour_close)     

    WeekDay_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
    #       Hour_open = h.text [ :h.text.find('-')-1]
    #       Hour_close = h.text [ h.text.find('-')+2 : ]
        WeekDay_list.append (s.text)
    WeekDay_list

    if 'Saturday' in WeekDay_list:
        saturday = 'Saturday'
    else:
        saturday = 'No' 

    if saturday == 'Saturday':
        saturday_close = Hour_close_list[4]
        return saturday_close
    else:
        saturday_close = 'Close'
        return saturday_close


#Extracting Sunday opening hour    
def Sunday_open():
    Hours_list = []
    Hoursportlet = soup.find_all('div', attrs = {'class' : 'p-locationfeatures2-main'})
    # print(Hoursportlet)
    Hour_open_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
        Hour_open = h.text [ :h.text.find('-')-1]
    #   Hour_close = h.text [ h.text.find('-')+2 : ]
        Hour_open_list.append (Hour_open)

    WeekDay_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
    #       Hour_open = h.text [ :h.text.find('-')-1]
    #       Hour_close = h.text [ h.text.find('-')+2 : ]
        WeekDay_list.append (s.text)
    WeekDay_list

    if 'Sunday' in WeekDay_list:
        sunday = 'Sunday'
    else:
        sunday = 'No' 

    if sunday == 'Sunday':
        sunday_open = Hour_open_list[6]
        return sunday_open
    else:
        sunday_open = 'Close'
        return sunday_open

    
#Extracting Sunday closing hour 
def Sunday_close():
    Hours_list = []
    Hoursportlet = soup.find_all('div', attrs = {'class' : 'p-locationfeatures2-main'})
    # print(Hoursportlet)
    Hour_close_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
        Hour_close = h.text [ h.text.find('-')+2 : ]
        Hour_close_list.append (Hour_close)     

    WeekDay_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
    #       Hour_open = h.text [ :h.text.find('-')-1]
    #       Hour_close = h.text [ h.text.find('-')+2 : ]
        WeekDay_list.append (s.text)
    WeekDay_list

    if 'Sunday' in WeekDay_list:
        sunday = 'Sunday'
    else:
        sunday = 'No' 

    if sunday == 'Sunday':
        sunday_close = Hour_close_list[5]
        return sunday_close
    else:
        sunday_close = 'Close'
        return sunday_close



# Location_Name(), Address_1(), Address_2(), City(), Province(), Postal_code(), Phone_number(), Fax(), URL(), Features(), Weekdays(), Hour_open(), Hour_close(), Services(), Side_Content()
# Monday_open(), Monday_close(), Tuesday_open(), Tuesday_close(), Wednesday_open(), Wednesday_close(), Thursday_open(), Thursday_close(), Friday_open(), Friday_close(), Saturday_open(), Saturday_close(), Sunday_open(), Sunday_close() 

In [25]:
 ############################ Loop in URL's and Creating the list to put in a Dataframe #####################################

Final_list_1 = []
Employee_ID = 0

 ########################################## Main For Loop on th list of URL's ################################################
for i in URL_column:
    #Reading the URL's (i) on the document:
    r = requests.get (i)
    soup = BeautifulSoup (r.text, 'html.parser')
    # print (r.text)
    General = soup.findAll( 'div' , attrs = {'class' : 'item text-center'})
    # print (General)

    
 ################################### creating the list with the clinic information ##########################################  
    row = [Location_Name(), 
           Address_1(), 
           Address_2(), 
           Postal_code(), 
           City(), 
           Province(),
           Phone_number(), 
           Fax(), 
           URL(), 
           Features(),
           Services(),
           'Logo_File_Name',
           Side_Content(), 
           Monday_open(), 
           Monday_close(), 
           Tuesday_open(),
           Tuesday_close(), 
           Wednesday_open(), 
           Wednesday_close(), 
           Thursday_open(), 
           Thursday_close(), 
           Friday_open(), 
           Friday_close(), 
           Saturday_open(), 
           Saturday_close(), 
           Sunday_open(), 
           Sunday_close()]
    
 ################################  Crossing the location with the Location_ID database #######################################
    Location_list = []
    Location_ID_list = []
    Location_list.append(Location_Name())
    Location_dataframe = pd.DataFrame(Location_list)
    Location_dataframe.columns = ['Location']
#     print (Location_dataframe)
    ID_Location = Location_dataframe.merge (Location_ID, left_on='Location', right_on='Location', how='left')
    ID_Location.set_index('Location', inplace = True , drop = True)
#     print (ID_Location)
    ID_location_final = ID_Location.loc [Location_Name(), 'Location_ID']
    ID_location_final_string = str(ID_location_final)
#     print (ID_location_final_string)


 ################################### Loop on the team portlet and creating the list ##########################################
    
    row_2_source = []
    

    Headshots = soup.find_all('div', attrs = {'class' : 'item text-center'})
#     print (Headshots)
    
    if Headshots == []:
        
        row_2_source = [None] * 200
        row_2 = row_2_source
        
    
    else:    
        for i in Headshots:
            
            img = i.find('img', alt=True)
            string1 = img['alt']
            First =  string1.find(',')
            Second = string1.find(',',First+1)
            Name = string1[ : First]                                                                  # First and Last Name
            First_Name = Name [ : Name.find(' ')]                                                     # First Name
            Last_Name = Name [Name.rfind(' ')+1 : ]                                                   # Last Name
            File_Name = First_Name + '-' + Last_Name + '-' + ID_location_final_string                 # File Name
            
        
            
            
            Job = string1[First+2 : Second]                                                           # Job String
            Job_list = []
            Job_list.append (Job)                                                                     # Job String in a list
            Job_Titles_df = pd.DataFrame(Job_list)                                                    # Job Title in a Dataframe
            Job_Titles_df.columns = ['CBI_Titles']
#             print (Job_Titles_df)
            Job_Title_SunLife = Job_Titles_df.merge (JT_CBI_SunLife, left_on='CBI_Titles', right_on='CBI_Titles', how='left')
            Job_Title_SunLife.set_index('CBI_Titles', inplace = True , drop = True)                   # Job Titles Crossed
            Job_Title_final = Job_Title_SunLife.loc [Job, 'SunLife_Titles']
             
        
#             print (Job_Title_SunLife)
            Bio = string1[Second+2 : ]
            if Job_Title_final != 'REVISE':                                                           # Adding Job Titles in a list when is usefull
                row_2_source.append (First_Name)
                row_2_source.append (Last_Name)
                row_2_source.append (File_Name)
                row_2_source.append (Job_Title_final)
                Employee_ID += 1
                row_2_source.append (Employee_ID)
                Missing_Columns = 200 - len(row_2_source) # creating the list with the clinic information
                Empty_rows = [None] * Missing_Columns
                row_2 = row_2_source + Empty_rows
            else:
                continue                                                                              # Disregarding the person if job title is not useful
        
#     print (row)
#     print (row_2)

            
 ################################### creating the list with the clinic information ##########################################
            



    final_row = row + row_2
    Final_list_1.append (final_row)

# Final_list_1


In [18]:
## Creating the dataframe and ading the list on it 

Dataframe = pd.DataFrame(Final_list_1)
Dataframe.columns = ['Location', 
                     'Address_1', 
                     'Address_2', 
                     'Postal Code', 
                     'City', 
                     'Province', 
                     'Phone', 
                     'Fax', 
                     'URL', 
                     'Amenities',
                     'Services',
                     'Logo Image File Name',
                     'Description',
                     'Monday_Open', 
                     'Monday_Close', 
                     'Tuesday_Open',
                     'Tuesday_Close',
                     'Wednesday_Open', 
                     'Wednesday_Close', 
                     'Thursday_Open',
                     'Thursday_Close',
                     'Friday_Open',
                     'Friday_Close',
                     'Saturday_Open', 
                     'Saturday_Close', 
                     'Sunday_Open',
                     'Sunday_Close',                     
                     'P1_First_Name', 'P1_Last_Name', 'P1_Image_File_Name', 'P1_Specialty','P1 License No.',
                     'P2_First_Name', 'P2_Last_Name', 'P2_Image_File_Name', 'P2_Specialty','P2 License No.',
                     'P3_First_Name', 'P3_Last_Name', 'P3_Image_File_Name', 'P3_Specialty','P3 License No.',
                     'P4_First_Name', 'P4_Last_Name', 'P4_Image_File_Name', 'P4_Specialty','P4 License No.',
                     'P5_First_Name', 'P5_Last_Name', 'P5_Image_File_Name', 'P5_Specialty','P5 License No.',
                     'P6_First_Name', 'P6_Last_Name', 'P6_Image_File_Name', 'P6_Specialty','P6 License No.',
                     'P7_First_Name', 'P7_Last_Name', 'P7_Image_File_Name', 'P7_Specialty','P7 License No.',
                     'P8_First_Name', 'P8_Last_Name', 'P8_Image_File_Name', 'P8_Specialty','P8 License No.',
                     'P9_First_Name', 'P9_Last_Name', 'P9_Image_File_Name', 'P9_Specialty','P9 License No.',
                     'P10_First_Name', 'P10_Last_Name', 'P10_Image_File_Name', 'P10_Specialty','P10 License No.',
                     'P11_First_Name', 'P11_Last_Name', 'P11_Image_File_Name', 'P11_Specialty','P11 License No.',
                     'P12_First_Name', 'P12_Last_Name', 'P12_Image_File_Name', 'P12_Specialty','P12 License No.',
                     'P13_First_Name', 'P13_Last_Name', 'P13_Image_File_Name', 'P13_Specialty','P13 License No.',
                     'P14_First_Name', 'P14_Last_Name', 'P14_Image_File_Name', 'P14_Specialty','P14 License No.',
                     'P15_First_Name', 'P15_Last_Name', 'P15_Image_File_Name', 'P15_Specialty','P15 License No.',
                     'P16_First_Name', 'P16_Last_Name', 'P16_Image_File_Name', 'P16_Specialty','P16 License No.',
                     'P17_First_Name', 'P17_Last_Name', 'P17_Image_File_Name', 'P17_Specialty','P17 License No.',
                     'P18_First_Name', 'P18_Last_Name', 'P18_Image_File_Name', 'P18_Specialty','P18 License No.',
                     'P19_First_Name', 'P19_Last_Name', 'P19_Image_File_Name', 'P19_Specialty','P19 License No.',
                     'P20_First_Name', 'P20_Last_Name', 'P20_Image_File_Name', 'P20_Specialty','P20 License No.',
                     'P21_First_Name', 'P21_Last_Name', 'P21_Image_File_Name', 'P21_Specialty','P21 License No.',
                     'P22_First_Name', 'P22_Last_Name', 'P22_Image_File_Name', 'P22_Specialty','P22 License No.',
                     'P23_First_Name', 'P23_Last_Name', 'P23_Image_File_Name', 'P23_Specialty','P23 License No.',
                     'P24_First_Name', 'P24_Last_Name', 'P24_Image_File_Name', 'P24_Specialty','P24 License No.',
                     'P25_First_Name', 'P25_Last_Name', 'P25_Image_File_Name', 'P25_Specialty','P25 License No.',
                     'P26_First_Name', 'P26_Last_Name', 'P26_Image_File_Name', 'P26_Specialty','P26 License No.',
                     'P27_First_Name', 'P27_Last_Name', 'P27_Image_File_Name', 'P27_Specialty','P27 License No.',
                     'P28_First_Name', 'P28_Last_Name', 'P28_Image_File_Name', 'P28_Specialty','P28 License No.',
                     'P29_First_Name', 'P29_Last_Name', 'P29_Image_File_Name', 'P29_Specialty','P29 License No.',
                     'P30_First_Name', 'P30_Last_Name', 'P30_Image_File_Name', 'P30_Specialty','P30 License No.',
                     'P31_First_Name', 'P31_Last_Name', 'P31_Image_File_Name', 'P31_Specialty','P31 License No.',
                     'P32_First_Name', 'P32_Last_Name', 'P32_Image_File_Name', 'P32_Specialty','P32 License No.',
                     'P33_First_Name', 'P33_Last_Name', 'P33_Image_File_Name', 'P33_Specialty','P33 License No.',
                     'P34_First_Name', 'P34_Last_Name', 'P34_Image_File_Name', 'P34_Specialty','P34 License No.',
                     'P35_First_Name', 'P35_Last_Name', 'P35_Image_File_Name', 'P35_Specialty','P35 License No.',
                     'P36_First_Name', 'P36_Last_Name', 'P36_Image_File_Name', 'P36_Specialty','P36 License No.',
                     'P37_First_Name', 'P37_Last_Name', 'P37_Image_File_Name', 'P37_Specialty','P37 License No.',
                     'P38_First_Name', 'P38_Last_Name', 'P38_Image_File_Name', 'P38_Specialty','P38 License No.',
                     'P39_First_Name', 'P39_Last_Name', 'P39_Image_File_Name', 'P39_Specialty','P39 License No.',
                     'P40_First_Name', 'P40_Last_Name', 'P40_Image_File_Name', 'P40_Specialty','P40 License No.']
Dataframe

,Location,Address_1,Address_2,Postal Code,City,Province,Phone,Fax,URL,Amenities,...,P39_First_Name,P39_Last_Name,P39_Image_File_Name,P39_Specialty,P39 License No.,P40_First_Name,P40_Last_Name,P40_Image_File_Name,P40_Specialty,P40 License No.
0,CBI Health Centre - Abbotsford,34194 Marshall Road,Suite 100,V2S 5E4,Abbotsford,BC,604.852.5655,604.852.5828,https://www.cbi.ca/web/physiotherapy-abbotsfor...,Free Parking ; Wheelchair Accessible,...,None,None,None,None,None,None,None,None,None,None
1,CBI Health Centre - Abbotsford Clearbrook,3106 Clearbrook Road,----,V2T 4N6,Abbotsford,BC,604.855.0155,604.855.0156,https://www.cbi.ca/web/physiotherapy-abbotsfor...,Free Parking ; Late Hours ; Wheelchair Access...,...,None,None,None,None,None,None,None,None,None,None
2,CBI Health Centre - Abbotsford Hand Therapy,34194 Marshall Road,Suite 100,V2S 5E4,Abbotsford,BC,604.859.4747,604.859.4767,https://www.cbi.ca/web/physiotherapy-abbotsfor...,,...,None,None,None,None,None,None,None,None,None,None
3,CBI Health Centre - Abbotsford West Railway,2375 West Railway Street,Unit 1,V2S 2E3,Abbotsford,BC,604.854.3545,604.853.8085,https://www.cbi.ca/web/physiotherapy-abbotsfor...,Free Parking ; Late Hours ; Wheelchair Access...,...,None,None,None,None,None,None,None,None,None,None
4,CBI Health Centre - Antigonish,40 Church Street,----,B2G 2C7,Antigonish,NS,902.863.5559,902.863.5979,https://www.cbi.ca/web/physiotherapy-antigonis...,Free Parking ; Late Hours ; Wheelchair Access...,...,None,None,None,None,None,None,None,None,None,None
5,CBI Health Centre - Barrie Quarry Ridge,34 Quarry Ridge Road,Suite 100,L4M 7G1,Barrie,ON,705.728.6060,705.728.7592,https://www.cbi.ca/web/physiotherapy-barrie-qu...,Free Parking ; Late Hours ; Wheelchair Access...,...,None,None,None,None,None,None,None,None,None,None
6,CBI Health Centre - Bedford,1550 Bedford Highway,Suite 100,B4A 1E6,Bedford,NS,902.423.6666,902.423.2998,https://www.cbi.ca/web/physiotherapy-bedford-b...,Free Parking ; Late Hours,...,None,None,None,None,None,None,None,None,None,None
7,CBI Health Centre - Belleville Millennium,81 Millennium Parkway,Suite 4A,K8N 4Z5,Belleville,ON,613.967.9675,613.967.9604,https://www.cbi.ca/web/physiotherapy-bellevill...,Free Parking ; Wheelchair Accessible,...,None,None,None,None,None,None,None,None,None,None


In [26]:
Dataframe.to_excel('Output_CBI_Sun_Life_before_email.xlsx')

In [24]:
############################################### Reading the Location ID's #####################################################
Loc_ID = pd.read_excel('CBI_Locations_ID.xlsx')
Location_ID = pd.DataFrame(Loc_ID)
# Location_ID

In [21]:
Dataframe_with_Loc_ID = Dataframe.merge(Location_ID, left_on='Location', right_on='Location', how='left')
# print (Dataframe_with_Loc_ID.columns)
# Dataframe_with_Loc_ID ['Location_ID', 'Email']
# Dataframe_with_Loc_ID.dtypes
# Dataframe_with_Loc_ID ['Location_ID'] = Dataframe_with_Loc_ID['Location_ID'].astype(int)
# Dataframe_with_Loc_ID ['Location_ID'].dtype


Dataframe_with_Loc_ID = Dataframe_with_Loc_ID [[
                     'Location_ID', 
                     'Location', 
                     'Address_1', 
                     'Address_2', 
                     'Postal Code', 
                     'City', 
                     'Province', 
                     'Phone', 
                     'Fax', 
                     'Location_Email',
                     'URL', 
                     'Amenities',
                     'Services',
                     'Logo Image File Name',
                     'Description',
                     'Monday_Open', 
                     'Monday_Close', 
                     'Tuesday_Open',
                     'Tuesday_Close',
                     'Wednesday_Open', 
                     'Wednesday_Close', 
                     'Thursday_Open',
                     'Thursday_Close',
                     'Friday_Open',
                     'Friday_Close',
                     'Saturday_Open', 
                     'Saturday_Close', 
                     'Sunday_Open',
                     'Sunday_Close',                     
                     'P1_First_Name', 'P1_Last_Name', 'P1_Image_File_Name', 'P1_Specialty','P1 License No.',
                     'P2_First_Name', 'P2_Last_Name', 'P2_Image_File_Name', 'P2_Specialty','P2 License No.',
                     'P3_First_Name', 'P3_Last_Name', 'P3_Image_File_Name', 'P3_Specialty','P3 License No.',
                     'P4_First_Name', 'P4_Last_Name', 'P4_Image_File_Name', 'P4_Specialty','P4 License No.',
                     'P5_First_Name', 'P5_Last_Name', 'P5_Image_File_Name', 'P5_Specialty','P5 License No.',
                     'P6_First_Name', 'P6_Last_Name', 'P6_Image_File_Name', 'P6_Specialty','P6 License No.',
                     'P7_First_Name', 'P7_Last_Name', 'P7_Image_File_Name', 'P7_Specialty','P7 License No.',
                     'P8_First_Name', 'P8_Last_Name', 'P8_Image_File_Name', 'P8_Specialty','P8 License No.',
                     'P9_First_Name', 'P9_Last_Name', 'P9_Image_File_Name', 'P9_Specialty','P9 License No.',
                     'P10_First_Name', 'P10_Last_Name', 'P10_Image_File_Name', 'P10_Specialty','P10 License No.',
                     'P11_First_Name', 'P11_Last_Name', 'P11_Image_File_Name', 'P11_Specialty','P11 License No.',
                     'P12_First_Name', 'P12_Last_Name', 'P12_Image_File_Name', 'P12_Specialty','P12 License No.',
                     'P13_First_Name', 'P13_Last_Name', 'P13_Image_File_Name', 'P13_Specialty','P13 License No.',
                     'P14_First_Name', 'P14_Last_Name', 'P14_Image_File_Name', 'P14_Specialty','P14 License No.',
                     'P15_First_Name', 'P15_Last_Name', 'P15_Image_File_Name', 'P15_Specialty','P15 License No.',
                     'P16_First_Name', 'P16_Last_Name', 'P16_Image_File_Name', 'P16_Specialty','P16 License No.',
                     'P17_First_Name', 'P17_Last_Name', 'P17_Image_File_Name', 'P17_Specialty','P17 License No.',
                     'P18_First_Name', 'P18_Last_Name', 'P18_Image_File_Name', 'P18_Specialty','P18 License No.',
                     'P19_First_Name', 'P19_Last_Name', 'P19_Image_File_Name', 'P19_Specialty','P19 License No.',
                     'P20_First_Name', 'P20_Last_Name', 'P20_Image_File_Name', 'P20_Specialty','P20 License No.',
                     'P21_First_Name', 'P21_Last_Name', 'P21_Image_File_Name', 'P21_Specialty','P21 License No.',
                     'P22_First_Name', 'P22_Last_Name', 'P22_Image_File_Name', 'P22_Specialty','P22 License No.',
                     'P23_First_Name', 'P23_Last_Name', 'P23_Image_File_Name', 'P23_Specialty','P23 License No.',
                     'P24_First_Name', 'P24_Last_Name', 'P24_Image_File_Name', 'P24_Specialty','P24 License No.',
                     'P25_First_Name', 'P25_Last_Name', 'P25_Image_File_Name', 'P25_Specialty','P25 License No.',
                     'P26_First_Name', 'P26_Last_Name', 'P26_Image_File_Name', 'P26_Specialty','P26 License No.',
                     'P27_First_Name', 'P27_Last_Name', 'P27_Image_File_Name', 'P27_Specialty','P27 License No.',
                     'P28_First_Name', 'P28_Last_Name', 'P28_Image_File_Name', 'P28_Specialty','P28 License No.',
                     'P29_First_Name', 'P29_Last_Name', 'P29_Image_File_Name', 'P29_Specialty','P29 License No.',
                     'P30_First_Name', 'P30_Last_Name', 'P30_Image_File_Name', 'P30_Specialty','P30 License No.',
                     'P31_First_Name', 'P31_Last_Name', 'P31_Image_File_Name', 'P31_Specialty','P31 License No.',
                     'P32_First_Name', 'P32_Last_Name', 'P32_Image_File_Name', 'P32_Specialty','P32 License No.',
                     'P33_First_Name', 'P33_Last_Name', 'P33_Image_File_Name', 'P33_Specialty','P33 License No.',
                     'P34_First_Name', 'P34_Last_Name', 'P34_Image_File_Name', 'P34_Specialty','P34 License No.',
                     'P35_First_Name', 'P35_Last_Name', 'P35_Image_File_Name', 'P35_Specialty','P35 License No.',
                     'P36_First_Name', 'P36_Last_Name', 'P36_Image_File_Name', 'P36_Specialty','P36 License No.',
                     'P37_First_Name', 'P37_Last_Name', 'P37_Image_File_Name', 'P37_Specialty','P37 License No.',
                     'P38_First_Name', 'P38_Last_Name', 'P38_Image_File_Name', 'P38_Specialty','P38 License No.',
                     'P39_First_Name', 'P39_Last_Name', 'P39_Image_File_Name', 'P39_Specialty','P39 License No.',
                     'P40_First_Name', 'P40_Last_Name', 'P40_Image_File_Name', 'P40_Specialty','P40 License No.']]


In [22]:
Dataframe_with_Loc_ID.set_index('Location_ID', inplace = True , drop = True)
Dataframe_with_Loc_ID

,Location,Address_1,Address_2,Postal Code,City,Province,Phone,Fax,Location_Email,URL,...,P39_First_Name,P39_Last_Name,P39_Image_File_Name,P39_Specialty,P39 License No.,P40_First_Name,P40_Last_Name,P40_Image_File_Name,P40_Specialty,P40 License No.
Location_ID,,,,,,,,,,,,,,,,,,,,,
1,CBI Health Centre - Abbotsford,34194 Marshall Road,Suite 100,V2S 5E4,Abbotsford,BC,604.852.5655,604.852.5828,abbotsford@cbi.ca,https://www.cbi.ca/web/physiotherapy-abbotsfor...,...,None,None,None,None,None,None,None,None,None,None
2,CBI Health Centre - Abbotsford Clearbrook,3106 Clearbrook Road,----,V2T 4N6,Abbotsford,BC,604.855.0155,604.855.0156,abbotsfordclearbrook@cbi.ca,https://www.cbi.ca/web/physiotherapy-abbotsfor...,...,None,None,None,None,None,None,None,None,None,None
3,CBI Health Centre - Abbotsford Hand Therapy,34194 Marshall Road,Suite 100,V2S 5E4,Abbotsford,BC,604.859.4747,604.859.4767,abbotsford@cbi.ca,https://www.cbi.ca/web/physiotherapy-abbotsfor...,...,None,None,None,None,None,None,None,None,None,None
4,CBI Health Centre - Abbotsford West Railway,2375 West Railway Street,Unit 1,V2S 2E3,Abbotsford,BC,604.854.3545,604.853.8085,sportandspine@cbi.ca,https://www.cbi.ca/web/physiotherapy-abbotsfor...,...,None,None,None,None,None,None,None,None,None,None
5,CBI Health Centre - Antigonish,40 Church Street,----,B2G 2C7,Antigonish,NS,902.863.5559,902.863.5979,antigonish@cbi.ca,https://www.cbi.ca/web/physiotherapy-antigonis...,...,None,None,None,None,None,None,None,None,None,None
6,CBI Health Centre - Barrie Quarry Ridge,34 Quarry Ridge Road,Suite 100,L4M 7G1,Barrie,ON,705.728.6060,705.728.7592,barrie@cbi.ca,https://www.cbi.ca/web/physiotherapy-barrie-qu...,...,None,None,None,None,None,None,None,None,None,None
7,CBI Health Centre - Bedford,1550 Bedford Highway,Suite 100,B4A 1E6,Bedford,NS,902.423.6666,902.423.2998,bedford@cbi.ca,https://www.cbi.ca/web/physiotherapy-bedford-b...,...,None,None,None,None,None,None,None,None,None,None
8,CBI Health Centre - Belleville Millennium,81 Millennium Parkway,Suite 4A,K8N 4Z5,Belleville,ON,613.967.9675,613.967.9604,bellevillephysiotherapy@cbi.ca,https://www.cbi.ca/web/physiotherapy-bellevill...,...,None,None,None,None,None,None,None,None,None,None


In [27]:
Dataframe_with_Loc_ID.to_excel('Output_CBI_Sun_Life_Final.xlsx')